In [1]:
from network.srcnn import srcnn

from keras.callbacks import ModelCheckpoint

import numpy
import math
import cv2
import time

from utility.h5data import h5DataRead

Using TensorFlow backend.


In [2]:
EPOCHS = 10
BATCH_SIZE = 128

SCALE = 4
FILENAME_TRAIN = "dataTrain_" + str(SCALE) + ".h5"
FILENAME_VALIDATE = "dataValidate_" + str(SCALE) + ".h5"
FILENAME_CHECKPOINT = "dataCheckPoint_" + str(SCALE) + ".h5"

dnn = srcnn()
dnn.layers = 3
dnn.filter = [128, 64, 1]
dnn.conv = [9, 5, 5]
dnn.inputShape = (32, 32, 1)
dnn.lr = 0.0003
dnn.in_train = True

In [3]:
tick1 = cv2.getTickCount()

In [4]:
model = dnn.network()
print(model.summary())
data, label = h5DataRead(FILENAME_TRAIN)
val_data, val_label = h5DataRead(FILENAME_VALIDATE)

checkpoint = ModelCheckpoint(FILENAME_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='min')

callbacks_list = [checkpoint]

model.fit(data, label, batch_size=BATCH_SIZE, validation_data=(val_data, val_label),
                callbacks=callbacks_list, shuffle=True, epochs=EPOCHS, verbose=1)

/home/jovyan/work/network/srcnn.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", padding="valid", input_shape=(32, 32, 1..., kernel_size=(9, 9), filters=128, kernel_initializer="glorot_uniform", use_bias=True)`
  activation='relu', padding='valid', bias=True, input_shape=self.inputShape))
/home/jovyan/work/network/srcnn.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", padding="same", kernel_size=(5, 5), filters=64, kernel_initializer="glorot_uniform", use_bias=True)`
  activation='relu', padding='same', bias=True))
/home/jovyan/work/network/srcnn.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="linear", padding="valid", kernel_size=(5, 5), filters=1, kernel_initializer="glorot_uniform", use_bias=True)`
  activation='linear', padding='valid', bias=True))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 128)       10496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        204864    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 1)         1601      
Total params: 217,217
Trainable params: 217,089
Non-trainable params: 128
_________________________________________________________________
None
Train on 3200 samples, validate on 750 samples
Epoch 1/10
3200/3200 [==============================] - 229s 71ms/step - loss: 0.0686 - mean_squared_error: 0.0686 - val_loss: 0.0036 - val_mean_squared_error: 0.0036

Epoch 00001: val_loss improved from inf to 0.0

In [5]:
score = model.evaluate(val_data, val_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.00568182758118
Test accuracy: 0.00568182911351


In [6]:
fileName = 'yayoi_' + str(SCALE) + "_" + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.h5'
model.save_weights(fileName)
print(fileName + ' saved')

yayoi_4_2018-07-13_025511.h5 saved


In [7]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("processed time: " + str(tick) + " ms")

processed time: 46 mins 13 secs 824 ms
